In [ ]:
from desdeo.problem.testproblems.single_objective import new_branin_function
from desdeo.emo.hooks.archivers import Archive
from desdeo.emo import algorithms, selection, termination, generator, crossover, mutation, scalar_selection
import polars as pl
import numpy as np
import plotly.graph_objects as go


def run_nsga2_with_mode(
    mode: str,
    pop_size: int = 100,
    n_generations: int = 100,
    constraint_symbol: str = "c_1",
    constraint_threshold: float = 2.0,
):
    """Run the NSGA-II style EA once for a given constraint-handling mode."""
    # ---- Problem ----
    problem = new_branin_function()

    # ---- Method options ----
    nsga2_options = algorithms.nsga2_options()

    # You can keep most defaults from nsga2_options, just overwrite what matters.
    # If in your original notebook you customized these further, you can copy those
    # lines here; the important thing for this example is the `mode`.
    nsga2_options.template.crossover = crossover.SimulatedBinaryCrossoverOptions(
        xover_probability=0.9,
        xover_distribution=20,
    )

    nsga2_options.template.mutation = mutation.BoundedPolynomialMutationOptions(
        mutation_probability=1.0 / len(problem.variables),
        distribution_index=20,
    )

    nsga2_options.template.generator = generator.LHSGeneratorOptions(
        n_points=pop_size,
    )

    nsga2_options.template.selection = (
        selection.SingleObjectiveConstrainedRankingSelectorOptions(
            target_objective_symbol="f_1",
            constraint_symbol=constraint_symbol,
            constraint_threshold=constraint_threshold,
            population_size=pop_size,
            mode=mode,
        )
    )

    nsga2_options.template.mate_selection = scalar_selection.TournamentSelectionOptions(name='TournamentSelection', tournament_size=2, winner_size=pop_size)

    nsga2_options.template.termination = (
        termination.MaxGenerationsTerminatorOptions(
            max_generations=n_generations,
        )
    )

    # ---- Construct solver ----
    solver, extras = algorithms.emo_constructor(
        emo_options=nsga2_options,
        problem=problem,
    )

    archive = Archive(problem=problem, publisher=extras.publisher)
    extras.publisher.auto_subscribe(archive)
    extras.publisher.register_topics(
        archive.provided_topics[archive.verbosity],
        archive.__class__.__name__,
    )

    # ---- Run optimization ----
    _ = solver()  # result object not strictly needed; archive holds all solutions

    # Full history of solutions as Polars DataFrame
    solutions = archive.solutions

    return solutions


def main():
    modes = ["relaxed", "baseline", "alternate"]

    fig_scatter = go.Figure()

    for mode in modes:
        to_plot = run_nsga2_with_mode(mode=mode)

        fig_scatter.add_trace(
            go.Scatter(
                x=to_plot["f_1"],
                y=to_plot["c_1"],
                mode="markers",
                name=f"{mode} (all)",
                opacity=0.6,
            )
        )

    fig_scatter.update_layout(
        title="f_1 vs c_1 for different modes",
        xaxis_title="f_1",
        yaxis_title="c_1",
    )

    fig_scatter.show()

main()

Running mode: relaxed
Running mode: baseline
Running mode: alternate
